In [1]:
# AOI = 'POLYGON ((-85.299088 40.339368, -85.332047 40.241477, -85.134979 40.229427, -85.157639 40.34146, -85.299088 40.339368))'
AOI = "MULTIPOLYGON (((-110.49367408906883 39.1005200348037, -110.20673076923077 39.1005200348037, -110.16573886639677 38.41732165423689, -110.43901821862349 38.41732165423689, -110.49367408906883 39.1005200348037)))"
START_DATE = "2020-05-01"
END_DATE = "2020-06-30"

REQUEST_ID = '0'

In [2]:
import os
import json
import time
import shutil
import rasterio
import rasterio.mask as riomask
import shapely
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pylab as plt

from utils import crop_raster, merge_tiles, get_tiles

import warnings
warnings.filterwarnings('ignore')

In [13]:
NAME = "Fields' boundaries"
NB_USER = os.getenv('NB_USER')
# NB_USER = "quantum"
BASE = os.getcwd()
DATA_DIR = f"/home/{NB_USER}/work/jupyter/Landcover"

LOAD_DIR = os.path.join(DATA_DIR, "satellite_imagery")
RESULTS_DIR = os.path.join(DATA_DIR, "results/landcover")

In [4]:
default_crs = 'EPSG:4326'
polygon = shapely.wkt.loads(AOI)
aoi_filename = f"{time.time()}_aoi.geojson"
gpd.GeoDataFrame(
    gpd.GeoSeries([polygon]),
    columns=["geometry"]).to_file(aoi_filename, driver="GeoJSON")
aoi = gpd.read_file(aoi_filename)

In [5]:
import urllib.request

sentinel_tiles_path = os.path.join(DATA_DIR, "sentinel2grid.geojson")
# sentinel_tiles_path = "sentinel2grid.geojson"
tile_idx = get_tiles(aoi_filename, sentinel_tiles_path)
year = int(START_DATE.split("-")[0])

files = []
data_dir = f"{DATA_DIR}/landcover_dataset"
os.makedirs(data_dir, exist_ok=True)

for tile_i in tile_idx.tileID:
    tile_i = tile_i if len(tile_i) < 4 else tile_i[:3]
    # https://lulctimeseries.blob.core.windows.net/lulctimeseriespublic/lc2018/40S_20180101-20190101.tif
    tile_url = f"https://lulctimeseries.blob.core.windows.net/lulctimeseriespublic/lc{year-1}/{tile_i}_{year-1}0101-{year}0101.tif"
    path = f"{data_dir}/{tile_i}_20210101-20220101.tif"
    if not os.path.exists(path):
        urllib.request.urlretrieve(tile_url, path)
    files.append(path)
    
files

['/home/quantum/work/jupyter/Landcover/landcover_dataset/12S_20210101-20220101.tif',
 '/home/quantum/work/jupyter/Landcover/landcover_dataset/12S_20210101-20220101.tif']

In [6]:
class_names = {
    1: "Water",
    2: "Trees",
    4: "Flooded vegetation",
    5: "Crops",
    7: "Built Area",
    8: "Bare ground",
    9: "Snow/Ice",
    10: "Clouds",
    11: "Rangeland"
}

In [7]:
if len(files) > 1:
    raster_path = merge_tiles(
        files, os.path.join(data_dir, "aoi_raster.tif"))
else:
    raster_path = files[0]

Start merging to /home/quantum/work/jupyter/Landcover/landcover_dataset/aoi_raster.tif
Copying color table from /home/quantum/work/jupyter/Landcover/landcover_dataset/12S_20210101-20220101.tif to new file.
Creating output file that is 59575P x 89711L.
Processing /home/quantum/work/jupyter/Landcover/landcover_dataset/12S_20210101-20220101.tif [1/2] : 0Using internal nodata values (e.g. 0) for image /home/quantum/work/jupyter/Landcover/landcover_dataset/12S_20210101-20220101.tif.
Copying nodata values from source /home/quantum/work/jupyter/Landcover/landcover_dataset/12S_20210101-20220101.tif to destination /home/quantum/work/jupyter/Landcover/landcover_dataset/aoi_raster.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
Processing /home/quantum/work/jupyter/Landcover/landcover_dataset/12S_20210101-20220101.tif [2/2] : 0Using internal nodata values (e.g. 0) for image /home/quantum/work/jupyter/Landcover/landcover_dataset/12S_20210101-20220101.tif.
...10...20...30...40...50

In [9]:
raster_path = crop_raster(raster_path, aoi_filename, raster_path.replace(".tif", "_crop.tif"))

with rasterio.open(raster_path) as src:
    img = src.read(1)
    mask = src.read_masks(1)
    profile = src.profile

In [10]:
NUM_CLASSES = len(class_names)
arr = np.array(range(0, NUM_CLASSES)) / NUM_CLASSES
colors = plt.cm.jet(arr)
colors[0] = (0,0,0,1)

labels = []

for label, name in enumerate(class_names):
#     print(label)
    class_area = len(img[img == label]) / 10 ** 4
    # convert list of float values into string representing color
    class_color = ",".join(list(map(lambda x: str(int(x)), colors[label][:-1] * 255)))
    if class_area != 0:
        labels.append({
            "color": class_color, 
            "name": class_names[name],
            "area": class_area
        })
    else:
        labels.append({
            "color": class_color, 
            "name": name
        })
labels = json.dumps(labels)
labels

'[{"color": "0,0,0", "name": "Water", "area": 338.723}, {"color": "0,0,254", "name": "Trees", "area": 0.0917}, {"color": "0,96,255", "name": "Flooded vegetation", "area": 0.1425}, {"color": "0,212,255", "name": 5}, {"color": "76,255,170", "name": "Built Area", "area": 0.127}, {"color": "170,255,76", "name": 8}, {"color": "255,229,0", "name": 9}, {"color": "255,122,0", "name": 10}, {"color": "254,18,0", "name": "Rangeland", "area": 155.558}]'

In [11]:
os.remove(aoi_filename)

In [14]:
# NB_USER = os.getenv('NB_USER')
os.makedirs(f"/home/{NB_USER}/work/results/landcover", exist_ok=True)
tile_name = f"{REQUEST_ID}_{START_DATE}_{END_DATE}.tif"

In [15]:
shutil.move(
    raster_path, f"/home/{NB_USER}/work/results/landcover/{tile_name}")

'/home/quantum/work/results/landcover/0_2020-05-01_2020-06-30.tif'